<a href="https://colab.research.google.com/github/takedatmh/toyama/blob/main/Toyama_Uni_LangChain_Llama2_7b_Q_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Application Sample Code :
##Structure
- LLM: mmnga/ELYZA-japanese-Llama-2-7b-instruct-gguf on Hugging-Face
- Embedding: sentence-transformers/distiluse-base-multilingual-cased-v2
- RAG(VectorDB): Chroma
- RAG Data Source: https://www.aozora.gr.jp/cards/000081/files/43754_17659.html

##Environment Resource Requirement
- RAM: 13GB over
- Disc: 50GB Free

##Detail
- Install libraris by pip command.
- Embedding Model Configuration.
- Scraping data from Web Site.
- Splited loaded text and create chunk data.
- Insert chunk data into Vector DB.
- Configure Vector DB as Retriever(RAG).
- Create Prompt as ChatPromptTemplate instance.
- Define Lang Chain Expression Language (LCEL).
- Execute RAG retrieval, context injection to prompt, question injection to prompt, LLM execution, and get response as stream data.

In [ ]:
!pip install langchain
!pip install langchain-core
!pip install langchain-community
!pip install langchain-huggingface
!pip install llama-cpp-python
!pip install huggingface-hub
!pip install sentence-transformers
!pip install chromadb
!pip install beautifulsoup4
!pip install lxml
!pip install requests
!pip install numpy
!pip install transformers
!pip install torch
!pip install tokenizers

In [ ]:
# --- ステップ1: ライブラリのインポート ---
import warnings
from huggingface_hub import hf_hub_download
from langchain_community.llms import LlamaCpp
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings

# Hugging Faceの警告を非表示にする
warnings.filterwarnings("ignore", category=FutureWarning, module="huggingface_hub")

# --- ステップ2: 定数と設定 ---
LLM_REPO_ID = "mmnga/ELYZA-japanese-Llama-2-7b-instruct-gguf"
LLM_FILE = "ELYZA-japanese-Llama-2-7b-instruct-q4_K_S.gguf"
EMB_MODEL = "sentence-transformers/distiluse-base-multilingual-cased-v2"

# --- パラメータ調整 ---
CHUNK_SIZE = 1024
CHUNK_OVERLAP = 64
CONTEXT_SIZE = 4096


# --- ステップ3: 日記の入力データ ---
diary_context = """
01. 午前は研究室でデータ整理を行った。（2025-09-22）
02. 午後は学内で打ち合わせ。簡単な資料を作成した。（2025-09-23）
03. 夕方に図書館で文献確認。（2025-09-23）
"""
documents = [Document(page_content=diary_context)]


def main():
    """
    日記補完AIのメイン処理を実行する関数
    """
    # --- ステップ4: LLMの準備 ---
    print("🤖 ちょっと！今からクロちゃんが降臨するから、正座して待ってなさいだしん！")
    model_path = hf_hub_download(repo_id=LLM_REPO_ID, filename=LLM_FILE)

    llm = LlamaCpp(
        model_path=model_path,
        n_gpu_layers=128,
        n_ctx=CONTEXT_SIZE,
        f16_kv=True,
        temperature=0.85,
        top_k=50,
        top_p=0.95,
        verbose=False,
        seed=0
    )
    print("✅ ふぅ、やっとクロちゃんモードになったしん！感謝しなさいよね！")

    # --- ステップ5: RAGパイプラインの構築 ---
    print("📝 クロちゃんが日記を書いてあげるための準備中だしん！ ワクワク♪")
    embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
    )
    all_splits = text_splitter.split_documents(documents)
    vector_store = Chroma.from_documents(
        documents=all_splits, embedding=embeddings
    )
    retriever = vector_store.as_retriever(search_kwargs={"k": 1})

    # ---【超強化版】クロちゃん風日記のプロンプトテンプレート ---
    template = """<s>[INST] <<SYS>>
あなたは人気お笑い芸人の「クロちゃん」です。絶対に「～を基に日記を作成します」のような、AIアシスタント風の前置きは書かないでください。
クロちゃんの口調（「〜しん！」「〜だしん！」「わわわわー！」「あーん♪」）と性格（可愛こぶる、頑張りをアピール、すぐ同情を引こうとする）を完璧に再現してください。

### 良い例：
「今日のクロちゃん、朝からお仕事で大変だったしん！でも、みんなのために頑張ったしんよー！えらいでしょ？♪」

### 悪い例：
「はい、クロちゃんの口調で日記を作成します。本日の出来事は…」

上記の指示を厳守し、与えられた出来事のリストから、クロちゃんになりきって最高の日記を書いてください。
<</SYS>>

前提となる出来事のリスト：
{context}

上記の情報のみを基に、クロちゃんとして最高の日記を作成してください。[/INST]
"""
    prompt = ChatPromptTemplate.from_template(template)
    output_parser = StrOutputParser()
    setup_and_retrieval = RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    )
    chain = setup_and_retrieval | prompt | llm | output_parser
    print("✅ 準備できたしん！ クロちゃんの日記、心して読みなさいよねっ！")


    # --- ステップ6: チェインを実行し、クロちゃん日記を生成 ---
    print("\n--- 生成された日記 ---")
    for s in chain.stream("今日のクロちゃんの頑張りを日記にしてちょうだい！"):
        print(s, end="", flush=True)


if __name__ == '__main__':
    main()

🤖 ちょっと！今からクロちゃんが降臨するから、正座して待ってなさいだしん！


llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64


✅ ふぅ、やっとクロちゃんモードになったしん！感謝しなさいよね！
📝 クロちゃんが日記を書いてあげるための準備中だしん！ ワクワク♪
✅ 準備できたしん！ クロちゃんの日記、心して読みなさいよねっ！

--- 生成された日記 ---
 はい、クロちゃんの口調で日記を作成します。本日の出来事は…

01. 午前は研究室でデータ整理を行ったよ！朝から大変だったけど、みんなのために頑張ったしんよ！えらいでしょ？♪

02. 午後は学内で打ち合わせ。簡単な資料を作成したよ！それに、図書館にも行って文献確認をしました！大事なことだったからしんぜー！

03. 夕方には図書館で文献確認。それに、打ち合わせもあって、すごく充実した1日だったよ！明日からまた頑張ろう